In [ ]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),  
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),  
    ('payments.csv', 'payments'),
    ('order_items.csv', 'order_items')  
]

# Connecting to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Priyansh@123',
    database='ecommerce'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/Priyansh/Desktop/OneDrive - BML MUNJAL UNIVERSITY/Data Analytics/Data Analytics/Datasets'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    try:
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)
        
        # Replace NaN with None to handle SQL NULL
        df = df.where(pd.notnull(df), None)
        
        # Debugging: Check for NaN values
        print(f"Processing {csv_file}")
        print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Generate the CREATE TABLE statement with appropriate data types
        columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
        cursor.execute(create_table_query)

        # Insert DataFrame data into the MySQL table
        for _, row in df.iterrows():
            # Convert row to tuple and handle NaN/None explicitly
            values = tuple(None if pd.isna(x) else x for x in row)
            sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(sql, values)

        # Commit the transaction for the current CSV file
        conn.commit()

    except Exception as e:
        print(f"Error processing file {csv_file}: {e}")

# Close the connection
conn.close()


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

# Connect to the MySQL database
db = mysql.connector.connect(
    host="localhost",
    username="root",
    password="Priyansh@123",
    database="ecommerce"
)

# Create a cursor object
cur = db.cursor()

# Basic Queries
1. List all unique cities where customers are located.
2. Count the number of orders placed in 2017.
3. Find the total sales per category.
4. Calculate the percentage of orders that were paid in installments.
5. Count the number of customers from each state. 





In [37]:
# answer 1
query = """select distinct customer_city from customers"""
cur.execute(query)
result = cur.fetchall()
df=pd.DataFrame(result,columns=["Cities"])
df

,Cities
0,franca
1,sao bernardo do campo
2,sao paulo
3,mogi das cruzes
4,campinas
...,...
4114,siriji
4115,natividade da serra
4116,monte bonito
4117,sao rafael


In [3]:
# answer 2
query = """select count(order_id) from orders where year(order_purchase_timestamp)= 2017"""
cur.execute(query)
result = cur.fetchall()
"total order in the year 2017 are :",result[0][0]

('total order in the year 2017 are :', 45101)

In [4]:
# answer 3
query = """
select upper(products.product_category )Category,
    round(sum(payments.payment_value),2) Sales 
        from products join order_items 
            on products.product_id=order_items.product_id 
                join payments on 
                    payments.order_id= order_items.order_id
                        group by category
                            order by Sales DESC"""
cur.execute(query)
result = cur.fetchall()

df=pd.DataFrame(result, columns=["Category", "Values"])
df_top_10 = df.head(10)


import plotly.express as px
fig = px.bar(df_top_10, x='Category', y='Values',
             title='Top 10 Product Categories by Sales',
             labels={'Category': 'Category', 'Values': 'Sales'},
             color='Values',
             color_continuous_scale=px.colors.sequential.Blues)  # Dark blue to light blue gradient

# Dark layout for adjusting the viewers perspective

fig.update_layout(
    xaxis_title='Category',
    yaxis_title='Sales',
    xaxis_tickangle=-45,
    plot_bgcolor='rgb(23, 25, 35)',  # Dark background for the plot area
    paper_bgcolor='rgb(10, 10, 20)',  # Darker background for the entire paper
    font=dict(color='white'),  # Set font color to white for better contrast
    coloraxis_colorbar=dict(
        title='Sales',
       
    )
)

# Show the plot
fig.show()

In [8]:
#answer 4
query = """select (sum(case when payment_installments >=1 then 1 else 0 end ) )/count(*)*100 from payments"""
cur.execute(query)
result = cur.fetchall()
"the customers that paid installments :",result[0][0]


Decimal('99.9981')

In [34]:
#answer 5 
query = """select customer_state as State ,count(customer_id) as Count 
            from customers 
                group by customer_state
                    order by Count Desc"""
cur.execute(query)
result = cur.fetchall()

df=pd.DataFrame(result,columns=["State","Count"])
print(df)



fig = px.bar(df, x='State', y='Count',
             title='Count of customer in each State ',
             labels={'state': 'State', 'count': 'count'},
             color='Count',
             color_continuous_scale=px.colors.sequential.Blues)  


fig.update_layout(
    xaxis_title='State',
    yaxis_title='Customer Count',
    xaxis_tickangle=-45,
    
    plot_bgcolor='rgb(23, 25, 35)',  
    paper_bgcolor='rgb(10, 10, 20)',  
    font=dict(color='white'),  
    coloraxis_colorbar=dict(
        title='Sales',
       
     ),
    width=900,  
    height=450
    
)

# Show the plot
fig.show()

   State  Count
0     SP  41746
1     RJ  12852
2     MG  11635
3     RS   5466
4     PR   5045
5     SC   3637
6     BA   3380
7     DF   2140
8     ES   2033
9     GO   2020
10    PE   1652
11    CE   1336
12    PA    975
13    MT    907
14    MA    747
15    MS    715
16    PB    536
17    PI    495
18    RN    485
19    AL    413
20    SE    350
21    TO    280
22    RO    253
23    AM    148
24    AC     81
25    AP     68
26    RR     46


# Intermediate Queries
1. Calculate the number of orders per month in 2018.
2. Find the average number of products per order, grouped by customer city.
3. Calculate the percentage of total revenue contributed by each product category.
4. Identify the correlation between product price and the number of times a product has been purchased.
5. Calculate the total revenue generated by each seller, and rank them by revenue.

In [47]:
#answer 1
query = """ select month(order_purchase_timestamp) month,count(order_id) as Number_of_orders
                    from orders where year(order_purchase_timestamp)=2018
                        group by month
                            order by Month asc """
cur.execute(query)
result = cur.fetchall()
df=pd.DataFrame(result,columns=["Month","Order Count"])

import calendar
df["Month"] = df["Month"].apply(lambda x: calendar.month_name[x])

print(df)


fig = px.bar(df, x='Month', y='Order Count',
             title='Count of Order per month for year 2018 ',
             labels={'Month': 'Month', 'count': 'Order Count'},
             color='Order Count',
             color_continuous_scale=px.colors.sequential.Blues)  


fig.update_layout(
    xaxis_title='Months',
    yaxis_title='Order Count',
    xaxis_tickangle=-45,
    
    plot_bgcolor='rgb(23, 25, 35)',  
    paper_bgcolor='rgb(10, 10, 20)',  
    font=dict(color='white'),  
    coloraxis_colorbar=dict(
        title='Order Count',
       
     ),
    width=800,  
    height=450
    
)

# Show the plot
fig.show()


       Month  Order Count
0    January         7269
1   February         6728
2      March         7211
3      April         6939
4        May         6873
5       June         6167
6       July         6292
7     August         6512
8  September           16
9    October            4


In [49]:

#ANSWER 2 Find the average number of products per order, grouped by customer city.

query = """ with count_per_order as 
(SELECT  orders.order_id , orders.customer_id ,count(order_items.order_id) as OC
from orders join order_items ON orders.order_id=order_items.order_id
GROUP BY orders.order_id , orders.customer_id) 

SELECT customers.customer_city ,round(avg(count_per_order.OC),2) Count_Order from customers JOIN count_per_order
ON customers.customer_id=count_per_order.customer_id
GROUP BY customers.customer_city
order BY Count_Order DESC; """
cur.execute(query)
result = cur.fetchall()
df=pd.DataFrame(result,columns=["City","AVG OrderCount "])
df

,City,AVG OrderCount
0,padre carvalho,7.00
1,celso ramos,6.50
2,candido godoi,6.00
3,datas,6.00
4,matias olimpio,5.00
...,...,...
4105,sebastiao leal,1.00
4106,morro agudo de goias,1.00
4107,santa filomena,1.00
4108,guamiranga,1.00


In [59]:
#3. Calculate the percentage of total revenue contributed by each product category.
query = """
SELECT 
    UPPER(products.product_category) AS category,
    ROUND(SUM(payments.payment_value) / (SELECT SUM(payment_value) FROM payments) * 100, 2) AS sales
FROM 
    products
JOIN 
    order_items ON products.product_id = order_items.product_id
JOIN 
    payments ON payments.order_id = order_items.order_id
GROUP BY 
    products.product_category
ORDER BY 
    sales DESC;
"""

cur.execute(query)
result = cur.fetchall()

df=pd.DataFrame(result,columns=["Product category","Precentage Sales"])
df

,Product category,Precentage Sales
0,BED TABLE BATH,10.70
1,HEALTH BEAUTY,10.35
2,COMPUTER ACCESSORIES,9.90
3,FURNITURE DECORATION,8.93
4,WATCHES PRESENT,8.93
...,...,...
69,HOUSE COMFORT 2,0.01
70,CDS MUSIC DVDS,0.01
71,PC GAMER,0.01
72,FASHION CHILDREN'S CLOTHING,0.00


In [66]:
import numpy as np 
#4. Identify the correlation between product price and the number of times a product has been purchased.
query="""select products.product_category, 
count(order_items.product_id),
round(avg(order_items.price),2)
from products join order_items
on products.product_id = order_items.product_id
group by products.product_category"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Category", "order_count","price"])
df

arr1=df["order_count"]
arr2=df["price"]
corr_coef = np.corrcoef(arr1, arr2)
print("the correlation is:",corr_coef[0][1])

the correlation is: -0.10631514167157562


As the value is -.106514 is very much close to 0 or can say is neutral as its not near to -1  , so no impact of orders on product and there exist a neutral relation . the correlation coeeficient is fro -1 to +1 , (-1 means inversely proportional and +1 means directly proportional)

In [117]:
#5. Calculate the total revenue generated by each seller, and rank them by revenue.
query = """select * , dense_rank() over (order by total_revenue desc) as Revenue_rank from
(SELECT 
    order_items.seller_id,
    round(SUM(payments.payment_value),2) AS total_revenue
FROM 
    order_items
JOIN 
    payments ON order_items.order_id = payments.order_id
GROUP BY 
    order_items.seller_id) as payment_value_table

"""


cur.execute(query)
data = cur.fetchall()

df=pd.DataFrame(data,columns=["Order_ID","Payment_Value","Ranking"])

print



df1=df.head(5)
fig = px.bar(df1, x='Order_ID', y='Payment_Value',
             title='Top Five Ranking pf Orders according to Payment Value  ',
             labels={'Order_ID': 'Order_ID', 'Payment_Value': 'Payment_Value'},
             color='Payment_Value',
             color_continuous_scale=px.colors.sequential.Blues)  


fig.update_layout(
    xaxis_title='Order ID',
    yaxis_title='Payment Value',
    xaxis_tickangle=-45,
    
    plot_bgcolor='rgb(23, 25, 35)',  
    paper_bgcolor='rgb(10, 10, 20)',  
    font=dict(color='white'),  
    coloraxis_colorbar=dict(
        title='Order Count',
       
     ),
    width=600,  
    height=500
    
)

# Show the plot
fig.show()

# Advanced Queries
1. Calculate the moving average of order values for each customer over their order history.
2. Calculate the cumulative sales per month for each year.
3. Calculate the year-over-year growth rate of total sales.
4. Calculate the retention rate of customers, defined as the percentage of customers who make another purchase within 6 months of their first purchase.
5. Identify the top 3 customers who spent the most money in each year.

In [96]:
#1. Calculate the moving average of order values for each customer over their order history.

query = """
SELECT 
    order_id,
    order_purchase_timestamp,
    payment,
    AVG(payment) OVER (PARTITION BY customer_id ORDER BY order_purchase_timestamp ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS mov_avg
FROM (
    SELECT 
        orders.order_id,
        orders.order_purchase_timestamp,
        payments.payment_value AS payment,
        orders.customer_id
    FROM 
        payments
    JOIN 
        orders ON orders.order_id = payments.order_id
) AS A
"""
cur.execute(query)
mov_avg = cur.fetchall()

df = pd.DataFrame(mov_avg, columns=["Order ID", "Order Purchase Timestamp", "Payment", "Moving Average"])


df

,Order ID,Order Purchase Timestamp,Payment,Moving Average
0,5f79b5b0931d63f1a42989eb65b9da6e,2017-11-14 16:08:26,114.74,114.739998
1,a44895d095d7e0702b6a162fa2dbeced,2017-07-16 09:40:32,67.41,67.410004
2,316a104623542e4d75189bb372bc5f8d,2017-02-28 11:06:43,195.42,195.419998
3,5825ce2e88d5346438686b0bba99e5ee,2017-08-16 13:09:20,179.35,179.350006
4,0ab7fb08086d4af9141453c91878ed7a,2018-04-02 13:42:17,107.01,107.010002
...,...,...,...,...
103881,814d6a3a7c0b32b2ad929ac6328124e9,2018-03-29 16:59:26,0.64,27.120001
103882,8c855550908247a7eff50281b92167a8,2018-05-22 13:36:02,63.13,63.130001
103883,83b5fc912b2862c5046555ded1483ae9,2018-06-13 16:57:05,214.13,214.130005
103884,d0e7be325a1c986babc4e1cdb91edc03,2017-09-02 11:53:32,45.50,45.500000


In [104]:
#2. Calculate the cumulative sales per month for each year.
query ="""

select years,months,payment ,sum(payment) over (order by years,months) cumulative_sales 
from (select year(orders.order_purchase_timestamp) as years , 
            month(orders.order_purchase_timestamp) as months ,
            round(sum(payments.payment_value),2)as payment from orders join payments on orders.order_id = payments.order_id 
            group by years ,months
            order by years,months asc) as A
        """
cur.execute(query)
results = cur.fetchall()

df=pd.DataFrame(results,columns=["Year","Month","Payment","cumulative payment"])
df

,Year,Month,Payment,cumulative payment
0,2016,9,252.24,252.24
1,2016,10,59090.48,59342.72
2,2016,12,19.62,59362.34
3,2017,1,138488.04,197850.38
4,2017,2,291908.01,489758.39
5,2017,3,449863.60,939621.99
6,2017,4,417788.03,1357410.02
7,2017,5,592918.82,1950328.84
8,2017,6,511276.38,2461605.22
9,2017,7,592382.92,3053988.14


In [105]:
#3. Calculate the year-over-year growth rate of total sales.
query = """
WITH a AS (
    SELECT 
        YEAR(orders.order_purchase_timestamp) AS years,
        ROUND(SUM(payments.payment_value), 2) AS payment
    FROM 
        orders
    JOIN 
        payments ON orders.order_id = payments.order_id
    GROUP BY 
        YEAR(orders.order_purchase_timestamp)
    ORDER BY 
        years
)

SELECT 
    years,
    ROUND(((payment - LAG(payment, 1) OVER (ORDER BY years)) / 
           LAG(payment, 1) OVER (ORDER BY years)) * 100, 2) AS percentage_change
FROM 
    a
"""

#the above formula depicts current year - previous year / previous year for the year over year growth
# lag is used for retrieving data from the preceding row
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["years", "yoy % growth"])
df

,years,yoy % growth
0,2016,NaN
1,2017,12112.703761
2,2018,20.000924


In [123]:
#4. Identify the top 3 customers who spent the most money in each year.
query = """
SELECT 
    years, 
    customer_id, 
    payment, 
    d_rank
FROM
(
    SELECT 
        YEAR(orders.order_purchase_timestamp) AS years,
        orders.customer_id,
        SUM(payments.payment_value) AS payment,
        DENSE_RANK() OVER (
            PARTITION BY YEAR(orders.order_purchase_timestamp)
            ORDER BY SUM(payments.payment_value) DESC
        ) AS d_rank
    FROM 
        orders
    JOIN 
        payments ON payments.order_id = orders.order_id
    GROUP BY 
        YEAR(orders.order_purchase_timestamp),
        orders.customer_id
) AS a
WHERE 
    d_rank <= 3;
"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["years","id","payment","rank"])


import plotly.express as px
# Assuming df is your DataFrame with columns 'id', 'payment', and 'years'
fig = px.bar(df, x="id", y="payment", color="years", barmode="group",
             labels={"id": "Customer ID", "payment": "Payment", "years": "Year"},
             title="Top 3 Customers by Payment per Year",
             color_continuous_scale=px.colors.sequential.Plasma)  # A balanced color scale

# Update layout with specific styling and dimensions
fig.update_layout(
    xaxis_title='Customer ID',
    yaxis_title='Payment',
    legend_title='Year',
    title_x=0.5,  # Center the title
    height=570,  # Set height in pixels
    width=900,    # Set width in pixels
    
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    
    # Set background colors
    plot_bgcolor='rgb(40, 44, 52)',  # Slightly lighter dark background for the plot area
    paper_bgcolor='rgb(30, 30, 40)',  # Darker background for the entire paper
    font=dict(color='white')  # Set font color to white for better contrast
)

# Show the plot
fig.show()

